<a href="https://colab.research.google.com/github/bobby-he/Neural_Tangent_Kernel/blob/master/notebooks/anaytic_kernels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/bobby-he/Neural_Tangent_Kernel.git

Cloning into 'Neural_Tangent_Kernel'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 685 (delta 73), reused 5 (delta 2), pack-reused 571
Receiving objects: 100% (685/685), 31.54 MiB | 25.45 MiB/s, done.
Resolving deltas: 100% (413/413), done.


In [0]:
from google.colab import files

In [0]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
import seaborn as sns
sns.set()
from collections import OrderedDict
import sys

from Neural_Tangent_Kernel.src.NTK_net import *
use_cuda = True if torch.cuda.is_available() else False


In [0]:
class NNGP(object):

    def __init__(self, deepArcCosine,X,Y,X_star):
      self.deepArcCosine = deepArcCosine
      self.X = X
      self.Y = Y
      self.X_star = X_star
      self.K = deepArcCosine.K(X,X).cpu().numpy()
      self.K_star = deepArcCosine.K(X_star,X).cpu().numpy()
      self.K_ss = deepArcCosine.K(X_star,X_star).cpu().numpy()
      self.L =  np.linalg.cholesky(self.K+0.00005 * np.eye(X.shape[0]))
      self.n_test = X_star.shape[0]
    
    
    def prior_std(self):
      return np.sqrt(np.diag(self.K_ss))
      
    def posterior_mean(self):
      Lk = np.linalg.solve(self.L, self.K_star.T) 
      mean_star = np.dot(Lk.T, np.linalg.solve(self.L, self.Y.cpu().numpy())).reshape((self.n_test,))
      return mean_star
      
    def posterior_std(self):
      Lk = np.linalg.solve(self.L,self.K_star.T) 
      s2 = np.diag(self.K_ss) - np.sum(Lk**2, axis=0)
      stdv = np.sqrt(s2)
      return(stdv)
      
    def get_K(self):
      return self.K
    
    def get_Kstar(self):
      return self.K_star
    
    def get_Kss(self):
      return self.K_ss

In [0]:
class NTK_analytic_pred(object):

    def __init__(self, deepArcCosine,X,Y,X_star,t,eta,K,K_star,K_ss):
      self.deepArcCosine = deepArcCosine
      self.t=t
      self.eta=eta
      self.X = X
      self.Y = Y
      self.X_star = X_star
      self.K = K
      self.K_star = K_star
      self.K_ss = K_ss
      self.n_test = X_star.shape[0]
      self.NTK_star = deepArcCosine.NTK(X_star,X).cpu().numpy()
      self.NTK_ss = deepArcCosine.NTK(X_star,X_star).cpu().numpy()
      self.NTK = deepArcCosine.NTK(X,X).cpu().numpy()

    def NTK_mean(self):
      L_NTK = np.linalg.cholesky(self.NTK+0.00005 * np.eye(X.shape[0]))
      L_help = np.linalg.solve(L_NTK, self.NTK_star.T)  # 4 x 100
      mat = np.eye(4)-scipy.linalg.expm(-self.eta*self.t*self.NTK)
      Y_transf = mat.dot(self.Y.cpu().numpy())
      meanNTK_star = np.matmul(L_help.T, np.linalg.solve(L_NTK, Y_transf)).reshape((self.n_test,))
      return meanNTK_star


    def NTK_std(self):
      inv = np.linalg.inv(self.NTK)
      mat = np.eye(4)-scipy.linalg.expm(-self.eta*self.t*self.NTK)
      Temp = np.matmul(self.NTK_star,np.matmul(inv,mat))
      A = np.matmul(Temp,self.K_star.T)
      B = np.matmul(inv,np.matmul(mat,self.NTK_star.T))
      s2 = np.diag(self.K_ss)-2*np.diag(A)+np.diag(np.matmul(Temp,np.matmul(self.K,B)))
      stdv_NTK = np.sqrt(s2)
      return stdv_NTK